In [2]:
import pandas as pd
import numpy as np
import os as os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.util import ngrams
import nltk
from arabica import arabica_freq
from unidecode import unidecode
import re
import plotly.express as px
import texthero as hero
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.naive_bayes import BernoulliNB
import spacy
import re
import nlp
import warnings
import statsmodels.api as sm
from statsmodels.formula.api import ols


warnings.simplefilter(action='ignore', category=FutureWarning)

2023-08-15 17:05:35.775795: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-15 17:05:36.034041: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-15 17:05:36.035034: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 17:05:37.701075: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [26]:
df = pd.read_csv('sentimento - base1.csv')

In [27]:
df.head()

,id_issue,mes_chat,SENTIMENTO,TIPO_MSG,hierarquia_4,ds_cluster_sociodemographic,ds_message
0,15388962,2023-04-01,neutro,duvida,Vencimento fatura,"0 - Mulher, Casada, 25 anos ou mais, cidade Pe...","['Que dia minha fatura fecha?','Ainda tenho dú..."
1,17497395,2023-07-01,neutro,duvida,Dúvidas sobre pagamento de fatura,"11 - Homem, Solteiro, Até 24 anos, cidade Pequ...","['Quero pagar os 100 reais que eu tirei','Mais..."
2,15213047,2023-04-01,neutro,solicitacao,Não reconhecimento de compra no crédito,"17 - Mulher, 25 a 40 anos, Sudeste e Sul","['Apareceu uma compra que eu não fiz','Dúvida ..."
3,17117301,2023-06-01,neutro,problema,Parcelamento de fatura,"8 - Mulher, Solteira, Até 24 anos, cidade Pequ...",['No meu limite está escrito que está utilizad...
4,17345996,2023-07-01,negativo,problema,Dúvidas PIX,"10 - Mulher, Solteira, Até 24 anos, cidade Peq...","['Pix','E merda o will nunca teve essas coisas..."


In [28]:
df['ds_message'][236]

"['Olá','Thays. Tudo bem?','Não','sem mais dúvidas','Muito obrigada ❤️❤️','Quero saber se tenho algum limite extra (adicional','emergencial)?','Aah','ok','Ok','fico no aguardo','Falar de outra coisa']"

In [29]:
def classif_emoji(text):
    '''
    Replace emojis with word emoji
    '''
    regrex_pattern = re.compile(pattern="["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                "]+", flags=re.UNICODE)
    return regrex_pattern.sub(r'emoji ', str(text))

def clean_text(x):
    x = unidecode(x)
    x = x.strip()
    x = x.lower()
    x = re.sub(r'\t', ' ', x)
    x = re.sub(r'http\S+', '', x)  # remove url
    x = re.sub(r'screenshot sent', 'screenshotsent', x)
    x = re.sub(r'will bank', 'willbank', x)
    x = re.sub(r'(nbsp\;|[rl]dquo\;|quot\;|acute\;|tilde\;|circ\;|grave\;|cedil\;|\&|\/)', '', x)  # replace special characters
    
    x = x.replace('&ccedil;','ç')
    x = x.replace('&atilde;;','ã')
    x = x.replace('&otilde;;','õ')
    x = re.sub(r'(\d{1,}|\!|\?|\,|\.|\\n|\:|\;|\`|\´|\-|\_)', ' ', x)  # replace punctuation and remove numbers
    x = x.replace('  ',' ')
    x = x.replace('\n','')
    x = x.strip()
    return x


In [30]:
len(df)

14906

In [36]:
# não vou remover emojis
df['ds_message_tratada'] = df['ds_message'].apply(clean_text)

In [37]:
df['ds_message_tratada'][132]

"['ja confirmou a compra obrigado pela informacao tenha um bom dia ' 'fiz uma compra na internet com o meu cartao virtual so que ainda nao debitou o valor da compra no limite do meu cartao virtual' 'falar de outra coisa']"

In [38]:
df['n_palavras'] = df['ds_message_tratada'].str.count(' ')

In [39]:
from LeIA import SentimentIntensityAnalyzer 


# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()
# Create a new column in the dataframe to store the sentiment scores
df["sentiment"] = 0.0

# Iterate over the rows of the dataframe
for index, row in df.iterrows():
  # Use the analyzer to predict the sentiment of the text
  scores = analyzer.polarity_scores(row["ds_message"])
  # Store the compound score in the new column
  df.at[index, "sentiment"] = scores["compound"]

In [40]:
df['sentiment']

0        0.0000
1       -0.4215
2       -0.2960
3        0.3400
4       -0.7003
          ...  
14901    0.0000
14902   -0.2244
14903    0.0000
14904    0.0000
14905    0.1007
Name: sentiment, Length: 14906, dtype: float64

In [41]:
df[['sentiment', 'SENTIMENTO']].groupby('SENTIMENTO').mean()

,sentiment
SENTIMENTO,
negativo,-0.359277
neutro,-0.107568
positivo,0.129828


In [57]:
df['sentiment'].describe()

count    14906.000000
mean        -0.153777
std          0.398074
min         -0.998800
25%         -0.421500
50%          0.000000
75%          0.000000
max          0.994100
Name: sentiment, dtype: float64

In [59]:
len(df.query('sentiment < -0.9'))

454

In [60]:
df.query('sentiment < -0.9').to_csv('muito_negativo_alg_leya.csv')

In [51]:
df['fx_sentiment'] = np.where(df.sentiment < 0, 'negativo', 
                     np.where(df.sentiment > 0, 'positivo', 'neutro'))

In [52]:
df['fx_sentiment'].value_counts()

negativo    7083
neutro      4918
positivo    2905
Name: fx_sentiment, dtype: int64

In [53]:
pd.crosstab(df.fx_sentiment, df.SENTIMENTO)

SENTIMENTO,negativo,neutro,positivo
fx_sentiment,,,
negativo,63,53,26
neutro,9,43,11
positivo,18,28,43


In [54]:
for i in df.query('fx_sentiment == "negativo" & SENTIMENTO == "positivo"').sort_values('sentiment', ascending = True).head(10)['ds_message']:
    print(i)

['Sim.','Entendi. Mas só me confirma o seguinte','como eu efetuei o pagamento','regularizei a situação com o banco antes dos 10 dias','não corre o risco do meu nome ficar negativado né?!','Falar de outra coisa','Foi o que fui informada pelo meu advogado','mas antes de tomar as medidas cabíveis eu vim entender a situação e saber se foi um erro mesmo e se está tudo ok com a minha situação.','Somente o e-mail.','Perfeito!','Minhas dúvidas era somente essas mesmo. Muito obrigada pelo excelente atendimento!','Claro','só um momento.','E eu recebi hoje essa informação do banco e não entendi.','Ok.','Certo Caroline.','Recebi essa mensagem e dizia que em caso de pagamento','devo desconsiderar.','Certo. Então tudo está correto?  Não corre o risco de meu nome ficar negativado?  Esse 5 dias úteis é a partir do momento que eu efetuei o pagamento ou que eu recebi o comunicado?','Esse ficou melhor?','Recebi um comunicado do Serasa que meu nome iria ser negativado pela fatura anterior','mas eu efetuei

In [56]:
for i in df.query('fx_sentiment == "negativo" & SENTIMENTO == "neutro"').sort_values('sentiment', ascending = True).head(10)['ds_message']:
    print(i)

['Sim','me ajudou','Mas enviamos o nome da referência que poderia entregar no mercadinho rocha','Sério','Nos botamos por botar pq aqui a maioria das casas são sem número','E como vai ficar a situação da minha entrega','Responder mensagem de antes','Porque como cliente estou muito insatisfeito','Enviei o maximo sim de informações','Quantos dias leva a revolução','Botei o endereço tudo explicado e devolveram novamente...','E mas quantos pra chegar ate mim','Constou como entrega não efetuada ','o tempo inteiro acompanhando ja fez mas de 30 dias','Já pedi minha 2ª via','???','O dia todo acompanhando o rastreio  olhe agora e esta dizendo que não entregou por falta de número do imóvel resolve meu problema por favor','Sim esse mesmo','Como vai ficar minha situação','Boa tarde alguma pessoa pra mim atender','Estou entrando em contato referente a entrega do meu cartão que pela segunda vez voltou sendo que me passaram o link de acompanhamento e estou acompanhando a risca inclusive ollhei ontem '

In [55]:
for i in df.query('fx_sentiment == "positivo" & SENTIMENTO == "negativo"').sort_values('sentiment', ascending = True).head(10)['ds_message']:
    print(i)

['Pelo aplicativo','Não consigo','Não já vi q não vão resolver','Por favor podem aumentar meu limite']
['👍','Vocês não falaram que conseguiram detectar o problema e que demora é essa pelo amor de Deus','Se conseguir detectar o problema já deu tempo suficiente para resolver não acha não','Misericórdia','Meu deus','Quero sabe da minha senha 😿','Mas resolve hj né','Pelo amor de Deus','Essa abençoada senha ?']
['Boa tarde e aí','Não demora muito porque tbm tenho muita coisa pra fazer OK 👍','Só 100','reais de limite 😏','Oiii','Falar de outra coisa','Qual e meu limite do cartão físico mesmo?','Quanto tenho de limite nó cartão físico pra começa a comprar?','Mais opções']
['Quase 1 hora e nada de resposta','Ainda preciso de ajuda.','Preciso pagar minhas parcelas  pra voltar a usar meu limite','Me ajuda','Q falta de respeito com o cliente','Consegui pagar pelo mês de maio','????','Pra responder','Mais de uma hora tentando resolver e vcs não me dão  nenhum  retorno','Mais o limite não foi libera

In [49]:
for i in df.query('sentiment < 0')[['sentiment', 'SENTIMENTO', 'ds_message']].sort_values('sentiment', ascending = True).head(10)['ds_message']:
    print(i)

['Pois então Taís é o seguinte eu não quero mais usar esse cartão eu atrasei a fatura do mês passado por causa de vocês e negligência de vocês porque esse cartão não chegou aqui para mim eu tava com saldo na conta para pagar a fatura não paguei porque o cartão não chegou para mim eu tô com saldo preso na minha conta sem conseguir movimentar porque eu não estou com cartão e agora eu não quero mais esse cartão eu quero dinheiro para legalizar minha minha dívida com vocês e sair fora e aí vocês vão resolver eu vou ter que procurar um advogado pagar esse valor para um advogado para poder estar resolvendo isso aí eu vou ter que ir nos meus direitos procurar','Quero saldo do meu saldo que tá na minha conta para poder completar pagar e sair fora','Segunda-feira eu vou procurar patrulha do consumidor vou procurar um advogado vou vou em tudo que for lugar mas eu vou porque eu não vou pagar um monte de juros desse jeito aí porque por causa de vocês porque não foi culpa minha porque eu tenho como

In [61]:
exemplo_base = df[df['SENTIMENTO'].notnull()]
len(exemplo_base)

294

In [62]:
stopwords_will= ['da', 'em', 'você', 'de', 'ao', 'os', 'que', 'e', 'eu', 'o', 'estou', 'ele','ela','pra', 'pro', 'entao', 'para', 'voce',
                 'pra', 'tá', 'né', 'ah', 'aí', 'dá', 'ó', 'mim', 'então', 'aqui', 'tô', 'pro',
            'seu', 'sua', 'por', 'sobre', 'mais', 'uma', 'um','como', 'meu', 'com', 'outro','outra','das','dos','foi', 'fiz','pelo','tem','mas',
            'este', 'esse','ta', 'to', 'ai', 'la', 'lo', 'lhe', 'ne','pra', 'tá', 'né', 'ah', 'aí', 'dá', 'ó', 'mim', 'então', 'aqui', 'tô', 'pro', 'isso']

In [72]:
exemplo_base['ds_message'] = exemplo_base['ds_message'].fillna("").apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords_will]))
## retirando sinais de pontuação
exemplo_base['ds_message'] = exemplo_base['ds_message'].str.replace('[^\w\s]','')
## colocando todas as letras das mensagens em letras minúsculas
exemplo_base['ds_message'] = exemplo_base['ds_message'].str.lower() 

## Análise de sentimento
vetorizador = CountVectorizer()

X = vetorizador.fit_transform(exemplo_base['ds_message'])

#X = np.append(X, exemplo_base['n_palavras'])


y = exemplo_base.SENTIMENTO

clf = BernoulliNB()
clf = clf.fit(X,y)

def sentimentos(df, ds_message_tratada):
    df['sentimento_predict'] = clf.predict(vetorizador.transform(df[ds_message_tratada].fillna('')))
    return df

/tmp/ipykernel_1276268/225718448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1276268/225718448.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_1276268/225718448.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [65]:
exemplo_base = sentimentos(exemplo_base, 'ds_message')

/tmp/ipykernel_1276268/1889439955.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [66]:
exemplo_base.head()

,id_issue,mes_chat,SENTIMENTO,TIPO_MSG,hierarquia_4,ds_cluster_sociodemographic,ds_message,ds_message_tratada,n_palavras,sentiment,fx_sentiment,sentimento_predict
0,15388962,2023-04-01,neutro,duvida,Vencimento fatura,"0 - Mulher, Casada, 25 anos ou mais, cidade Pe...",dia minha fatura fechaainda tenho dúvidas,['que dia minha fatura fecha ' 'ainda tenho du...,8,0.0000,neutro,neutro
1,17497395,2023-07-01,neutro,duvida,Dúvidas sobre pagamento de fatura,"11 - Homem, Solteiro, Até 24 anos, cidade Pequ...",quero pagar 100 reais tireimais opçõesfalar co...,['quero pagar os reais que eu tirei' 'mais op...,52,-0.4215,negativo,neutro
2,15213047,2023-04-01,neutro,solicitacao,Não reconhecimento de compra no crédito,"17 - Mulher, 25 a 40 anos, Sudeste e Sul",apareceu compra não fizdúvida compra2703 5740 ...,['apareceu uma compra que eu nao fiz' 'duvida ...,14,-0.2960,negativo,neutro
3,17117301,2023-06-01,neutro,problema,Parcelamento de fatura,"8 - Mulher, Solteira, Até 24 anos, cidade Pequ...",no limite está escrito está utilizado 29308 sa...,['no meu limite esta escrito que esta utilizad...,41,0.3400,positivo,neutro
4,17345996,2023-07-01,negativo,problema,Dúvidas PIX,"10 - Mulher, Solteira, Até 24 anos, cidade Peq...",pixe merda will nunca teve essas coisasestou p...,['pix' 'e merda o will nunca teve essas coisas...,26,-0.7003,negativo,neutro


In [73]:
exemplo_base.sentimento_predict.value_counts()

neutro      221
negativo     42
positivo     31
Name: sentimento_predict, dtype: int64

In [68]:
pd.crosstab(exemplo_base.sentimento_predict, exemplo_base.SENTIMENTO)

SENTIMENTO,negativo,neutro,positivo
sentimento_predict,,,
negativo,42,0,0
neutro,48,124,49
positivo,0,0,31


In [74]:
for i in exemplo_base.query('sentimento_predict == "neutro" & SENTIMENTO == "negativo"').head(10)['ds_message']:
    print(i)

pixe merda will nunca teve essas coisasestou precisando fazer pix não conseguindodesde ontemnão fazer pixmais opções
já passou do prazo não recebique possa me atendetem alguém aipra me atenderalguém
falar coisaquero saber não entrou ainda reembolso shen
quanto tempo estorno cairestorno na faturaproblema comprafatura do cartãoesse tempo já passoutô tentando manda print só q ñ certomostrar exemplo
pixtodo dia problema nesse aplicativo deus maisfalar coisanão fazer pix
mudar endereço entregaresponder mensagem antestá processo se entrega nunca chegafalar coisafalta respeito clienteesse atendimento vcs péssimo
ok vou aguardarcancelar essa conta pois tive limite bloqueado falta pagamentojá paguei minha dívida não liberaram crédito quero cancelar a conta cartãofalar coisa
falar coisacartão físicono momento nãoobrigadojá faz 5 diasque diz cartão produçãoquero saber acontecendo
quero pagar a faturaé erro pagamentoeu depositei dinheiro não consigo pagar a faturavai demorar
ajudaoiainda aína verd

In [78]:
#Modelo de duvida, problema ou solicitacao

y = exemplo_base.TIPO_MSG

clf2 = BernoulliNB()
clf2 = clf2.fit(X,y)

def tipo_msg(df, ds_message_tratada):
    df['tipo_msg_predict'] = clf2.predict(vetorizador.transform(df[ds_message_tratada].fillna('')))
    return df

exemplo_base = tipo_msg(exemplo_base, 'ds_message')


/tmp/ipykernel_1276268/3680250037.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [79]:
exemplo_base.tipo_msg_predict.value_counts()

duvida         203
problema        73
solicitacao     18
Name: tipo_msg_predict, dtype: int64

In [80]:
pd.crosstab(exemplo_base.tipo_msg_predict, exemplo_base.TIPO_MSG)

TIPO_MSG,duvida,problema,solicitacao
tipo_msg_predict,,,
duvida,108,45,50
problema,0,69,4
solicitacao,0,0,18


In [81]:
for i in exemplo_base.query('tipo_msg_predict == "duvida" & TIPO_MSG == "problema"').head(10)['ds_message']:
    print(i)

no limite está escrito está utilizado 29308 saldo disponível é 6792não entendi valor 29308 porque paguei já 2 meses 200 no cartão wil banckjá paguei tenho dúvidas
entãomeu limite negativo ainda
já passou do prazo não recebique possa me atendetem alguém aipra me atenderalguém
falar coisaquero saber não entrou ainda reembolso shen
pixtodo dia problema nesse aplicativo deus maisfalar coisanão fazer pix
pixfatura do cartãonão cadastrar chavevoltar começo
ainda preciso ajudacertoentendi obrigadaparcelei minha fatura já paguei saldo conta não voltou limite ainda
nao eatou conseguindo enviar pixme diga
ok vou aguardarcancelar essa conta pois tive limite bloqueado falta pagamentojá paguei minha dívida não liberaram crédito quero cancelar a conta cartãofalar coisa
falar coisacartão físicono momento nãoobrigadojá faz 5 diasque diz cartão produçãoquero saber acontecendo


## Próximos passos

- Focar no preenchimento dos casos mapeados pelo Leya como muito negativos (checar)
- Procurar os casos com palavras de baixo calão
- Procurar casos "já faz", "problema" para acerto da classificação de problemas
- Avaliar se em alguns casos a classificação de sentimento negativo não seria neutro de fato (clientes que relatam demora ou que passou do prazo seria um fato objetivo ou escondem um stress?)
- Testar classificador usando um embedding ao invés de vetorizer
- Incluir nr de palavras e outras característias semânticas